In [3]:
from file_management import *
from helpers import *
import pandas as pd

In [4]:
ids, predictions, data, features = get_train_higgs_data()

In [5]:
df = pd.DataFrame(index=ids.flatten(), data=data, columns=features.flatten())
df_NaN = df.replace(-999, np.NaN)
df_NaN.head(10)

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
100000,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,41.928,197.760,...,-0.277,258.733,2.0,67.435,2.150,0.444,46.062,1.240,-2.475,113.497
100001,160.937,68.768,103.235,48.146,NaN,NaN,NaN,3.473,2.078,125.157,...,-1.916,164.546,1.0,46.226,0.725,1.158,NaN,NaN,NaN,46.226
100002,NaN,162.172,125.953,35.635,NaN,NaN,NaN,3.148,9.336,197.814,...,-2.186,260.414,1.0,44.251,2.053,-2.028,NaN,NaN,NaN,44.251
100003,143.905,81.417,80.943,0.414,NaN,NaN,NaN,3.310,0.414,75.968,...,0.060,86.062,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000
100004,175.864,16.915,134.805,16.405,NaN,NaN,NaN,3.891,16.405,57.983,...,-0.871,53.131,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000
100005,89.744,13.550,59.149,116.344,2.636,284.584,-0.540,1.362,61.619,278.876,...,2.237,282.849,3.0,90.547,-2.412,-0.653,56.165,0.224,3.106,193.660
100006,148.754,28.862,107.782,106.130,0.733,158.359,0.113,2.941,2.545,305.967,...,-1.443,294.074,2.0,123.010,0.864,1.450,56.867,0.131,-2.767,179.877
100007,154.916,10.418,94.714,29.169,NaN,NaN,NaN,2.897,1.526,138.178,...,-1.761,187.299,1.0,30.638,-0.715,-1.724,NaN,NaN,NaN,30.638
100008,105.594,50.559,100.989,4.288,NaN,NaN,NaN,2.904,4.288,65.333,...,0.024,129.804,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000
100009,128.053,88.941,69.272,193.392,NaN,NaN,NaN,1.609,28.859,255.123,...,0.845,294.741,1.0,167.735,-2.767,-2.514,NaN,NaN,NaN,167.735


In [6]:
df_NaN.columns

Index(['DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis',
       'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
       'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
       'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality',
       'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi',
       'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi',
       'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt',
       'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt'],
      dtype='object')

In [11]:
jet_num = df_NaN.get('PRI_jet_num')
unique, counts = np.unique(jet_num, return_counts=True)
unique, counts

(array([0., 1., 2., 3.]), array([99913, 77544, 50379, 22164]))

In [31]:
jet_zero = df_NaN[df_NaN['PRI_jet_num'] == 0.0]
jet_one = df_NaN[df_NaN['PRI_jet_num'] == 1.0]
jet_two = df_NaN[df_NaN['PRI_jet_num'] == 2.0]
jet_three = df_NaN[df_NaN['PRI_jet_num'] == 3.0]

In [23]:
jet_zero

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
100003,143.905,81.417,80.943,0.414,NaN,NaN,NaN,3.310,0.414,75.968,...,0.060,86.062,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
100004,175.864,16.915,134.805,16.405,NaN,NaN,NaN,3.891,16.405,57.983,...,-0.871,53.131,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
100008,105.594,50.559,100.989,4.288,NaN,NaN,NaN,2.904,4.288,65.333,...,0.024,129.804,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
100010,NaN,86.240,79.692,27.201,NaN,NaN,NaN,2.338,27.201,81.734,...,0.688,250.178,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
100013,82.488,31.663,64.128,8.232,NaN,NaN,NaN,2.823,8.232,58.649,...,1.433,163.420,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349990,127.822,82.865,100.024,0.860,NaN,NaN,NaN,3.210,0.860,71.325,...,2.674,120.320,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
349995,NaN,71.989,36.548,5.042,NaN,NaN,NaN,1.392,5.042,55.892,...,2.859,144.665,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
349996,NaN,58.179,68.083,22.439,NaN,NaN,NaN,2.585,22.439,50.618,...,-0.867,80.408,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
349998,94.951,19.362,68.812,13.504,NaN,NaN,NaN,3.365,13.504,55.859,...,0.811,112.718,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [24]:
for column in jet_zero:
    nan_percent = (jet_zero[column].isna().sum() / jet_zero[column].shape[0]) * 100
    if nan_percent > 0:
        print(column, ':' ,nan_percent)

DER_mass_MMC : 26.145746799715752
DER_deltaeta_jet_jet : 100.0
DER_mass_jet_jet : 100.0
DER_prodeta_jet_jet : 100.0
DER_lep_eta_centrality : 100.0
PRI_jet_leading_pt : 100.0
PRI_jet_leading_eta : 100.0
PRI_jet_leading_phi : 100.0
PRI_jet_subleading_pt : 100.0
PRI_jet_subleading_eta : 100.0
PRI_jet_subleading_phi : 100.0


In [38]:
df_no_nan = jet_zero.drop(['DER_deltaeta_jet_jet',
                         'DER_mass_jet_jet',
                         'DER_prodeta_jet_jet',
                         'DER_lep_eta_centrality',
                         'PRI_jet_leading_pt',
                         'PRI_jet_leading_eta',
                         'PRI_jet_leading_phi',
                         'PRI_jet_subleading_pt',
                         'PRI_jet_subleading_eta',
                         'PRI_jet_subleading_phi',
                         'PRI_jet_num',
                         'PRI_jet_all_pt'], axis=1)   

In [39]:
df_no_nan.fillna(jet_zero.mean(), inplace=True)
df_no_nan

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet
100003,143.905000,81.417,80.943,0.414,3.310,0.414,75.968,2.354,-1.285,22.647,-1.655,0.010,53.321,-0.522,-3.100,31.082,0.060,86.062
100004,175.864000,16.915,134.805,16.405,3.891,16.405,57.983,1.056,-1.385,28.209,-2.197,-2.231,29.774,0.798,1.569,2.723,-0.871,53.131
100008,105.594000,50.559,100.989,4.288,2.904,4.288,65.333,0.675,-1.366,39.008,2.433,-2.532,26.325,0.210,1.884,37.791,0.024,129.804
100010,120.667654,86.240,79.692,27.201,2.338,27.201,81.734,1.750,-1.412,29.718,-0.866,2.878,52.016,0.126,-1.288,51.276,0.688,250.178
100013,82.488000,31.663,64.128,8.232,2.823,8.232,58.649,1.303,-1.414,25.470,-0.654,-2.990,33.179,-1.665,-0.354,12.439,1.433,163.420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349990,127.822000,82.865,100.024,0.860,3.210,0.860,71.325,2.322,-1.321,21.474,-0.640,1.618,49.852,1.391,-0.867,35.847,2.674,120.320
349995,120.667654,71.989,36.548,5.042,1.392,5.042,55.892,1.258,-1.414,24.754,-0.414,-0.905,31.137,-0.950,0.380,46.520,2.859,144.665
349996,120.667654,58.179,68.083,22.439,2.585,22.439,50.618,1.162,-1.345,23.416,-1.609,2.776,27.202,0.308,1.042,46.737,-0.867,80.408
349998,94.951000,19.362,68.812,13.504,3.365,13.504,55.859,0.999,1.414,27.944,-2.211,2.792,27.915,-0.874,-0.296,12.150,0.811,112.718


In [41]:
np.var(df_no_nan, axis=0)

DER_mass_MMC                   1977.840960
DER_mass_transverse_met_lep    1024.217036
DER_mass_vis                   1447.306284
DER_pt_h                        278.043070
DER_deltar_tau_lep                0.480653
DER_pt_tot                      278.043068
DER_sum_pt                      555.117837
DER_pt_ratio_lep_tau              0.338250
DER_met_phi_centrality            0.877411
PRI_tau_pt                      231.936583
PRI_tau_eta                       1.521335
PRI_tau_phi                       3.302733
PRI_lep_pt                      212.748653
PRI_lep_eta                       1.718324
PRI_lep_phi                       3.304523
PRI_met                         411.864257
PRI_met_phi                       3.279712
PRI_met_sumet                  2818.158900
dtype: float64